In [47]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
token = spotipy.util.prompt_for_user_token('Julien Salomon',
                           client_id='3f0268c00c604e70aace4982367723c7',
                           client_secret='c9a4a4f2699642c795cb82595c2c8035',
                            redirect_uri='http://localhost:8888')
spotify = spotipy.Spotify(auth=token)
spotify.trace = False # turn on tracing
spotify.trace_out = False # turn on trace out

In [48]:
all_songs = pd.read_csv('../data/final_df.csv')
all_songs[['loudness']] =  0.496000 - all_songs[['loudness']]
features = ["acousticness", "danceability","energy","loudness", "tempo","valence"]
normalized_songs = all_songs[['artist', 'name_track', 'name_album'] + features]
normalized_songs[features] = normalized_songs[features] / all_songs[features].max()
normalized_songs

/Users/jsalomon/Anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,artist,name_track,name_album,acousticness,danceability,energy,loudness,tempo,valence
0,Frank Sinatra,Nice 'N' Easy - Remastered,Sinatra Sings Alan & Marilyn Bergman,0.881288,0.497404,0.224261,0.397076,0.525406,0.480612
1,Frank Sinatra,Ol' Macdonald,Sinatra Sings Alan & Marilyn Bergman,0.742455,0.631360,0.414883,0.311886,0.405033,0.744898
2,Frank Sinatra,Sentimental Baby,Sinatra Sings Alan & Marilyn Bergman,0.904427,0.301142,0.148828,0.509874,0.393315,0.290816
3,Frank Sinatra,Love Looks So Well On You,Sinatra Sings Alan & Marilyn Bergman,0.906439,0.210800,0.292559,0.364372,0.408479,0.205102
4,Frank Sinatra,Sleep Warm,Sinatra Sings Alan & Marilyn Bergman,0.934608,0.151610,0.177370,0.521930,0.420585,0.141837
...,...,...,...,...,...,...,...,...,...
1939,Eminem,Curtains Up,Encore (Deluxe Version),0.482897,0.317757,0.738022,0.642588,0.549235,0.027449
1940,Eminem,Encore/Curtains Down,Encore (Deluxe Version),0.084306,0.628245,0.761468,0.202151,0.831339,0.576531
1941,Eminem,Curtains Close,The Eminem Show,0.521127,0.593977,0.419980,0.690046,0.474637,0.698980
1942,Eminem,Bitch,The Slim Shady LP (Expanded Edition),0.995976,0.866044,0.349643,0.542878,0.464491,0.988776


### Sinatra has recorded 2 Beatles songs

In [56]:
yesterdays = normalized_songs[normalized_songs.name_track.str.contains("Yesterday - Remastered 2015")]

In [57]:
uris = [track['uri'] for track in spotify.album(spotify.search("My Way", type="album")['albums']['items'][2]['uri'])['tracks']['items']]

In [58]:
for uri in uris:
    print(spotify.track(uri)['name'])
yesterday_uri = uris[3]

Watch What Happens
Didn't We
Hallelujah, I Love Her So
Yesterday
All My Tomorrows
My Way
A Day In The Life Of A Fool
For Once In My Life
If You Go Away
Mrs. Robinson
For Once In My Life - Rehearsal
My Way - Live At The Reunion Arena/1987


In [60]:
sinatra_yesterday = spotify.audio_features(yesterday_uri)[0]
sinatra_yesterday['artist'] = 'Frank Sinatra'
sinatra_yesterday['name_track'] = 'Yesterday'
sinatra_yesterday['loudness'] = 0.496000 - sinatra_yesterday['loudness']
for feature in features:
    sinatra_yesterday[feature] /= all_songs[feature].max()

In [61]:
yesterdays = yesterdays.append(sinatra_yesterday, ignore_index=True).dropna(axis=1)

In [62]:
duo = yesterdays.append(normalized_songs[normalized_songs.name_track.str.contains("Something")].iloc[[1,5]])
duo = duo.dropna(axis=1)
duo.name_track = duo.apply(lambda row: row['name_track'].replace(" - Remastered 2015", ""), axis=1)
duo[duo.name_track == "Yesterday"][['artist'] + features].set_index('artist').transpose().reset_index().rename(columns={'index': 'group'}).to_csv("../data/yesterday_sin_eminem.csv")
duo[duo.name_track == "Something"][['artist'] + features].set_index('artist').transpose().reset_index().rename(columns={'index': 'group'}).to_csv("../data/something_sin_eminem.csv")

/Users/jsalomon/Anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


### Rihanna and Eminem did tracks together

In [13]:
rihanna_eminem = normalized_songs[normalized_songs.name_track.str.contains("Love The Way")].iloc[[1,4]]
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.name_track.str.contains("The Monster")].iloc[0])
rihanna_eminem = rihanna_eminem.append(normalized_songs[normalized_songs.name_track.str.contains("Numb")].iloc[0])

In [14]:
rihanna_eminem.set_index('name_track')[features].transpose().reset_index().rename(columns={'index': 'group'}).to_csv('../data/rihanna-eminem.csv')

### MJ and Sinatra ft Quincy Jones

In [15]:
normalized_songs[normalized_songs.artist == 'Michael Jackson'].groupby('artist').mean().reset_index().append(normalized_songs[normalized_songs.name_track.str.contains('Just Lose')].iloc[0]).to_csv('mj-eminem.csv')#.plot.bar(figsize=(20,20))

In [16]:
normalized_songs[normalized_songs.name_track.str.contains('Til You')].append(normalized_songs[normalized_songs.name_track.str.contains('Stop The Music')]).iloc[[0, 3]].to_csv('rihanna_mj.csv')

In [66]:
mj_quincy = pd.DataFrame()
mj_quincy['mean'] = normalized_songs[(normalized_songs.name_album == 'Off the Wall') | (normalized_songs.name_album == 'Thriller') | (normalized_songs.name_album == 'Bad (Remastered)')].mean()
mj_quincy['std'] = normalized_songs[(normalized_songs.name_album == 'Off the Wall') | (normalized_songs.name_album == 'Thriller') | (normalized_songs.name_album == 'Bad (Remastered)')].std()
mj_quincy['ci_ub'] = mj_quincy['mean'] + mj_quincy['std']
mj_quincy['ci_lb'] = mj_quincy['mean'] - mj_quincy['std']
mj_quincy.reset_index().rename(columns={'index':'group'}).to_csv("../data/mj_quincy.csv")

In [67]:
sinatra_quincy = pd.DataFrame()
sinatra_quincy['mean'] = normalized_songs[normalized_songs.name_album.str.contains('L.A')].mean()
sinatra_quincy['std'] = normalized_songs[normalized_songs.name_album.str.contains('L.A')].std()
sinatra_quincy['ci_ub'] = sinatra_quincy['mean'] + sinatra_quincy['std']
sinatra_quincy['ci_lb'] = sinatra_quincy['mean'] - sinatra_quincy['std']
sinatra_quincy.rename(columns={'index':'group'}).to_csv("../data/sin_quincy.csv")

### Mean and STD for all artists

In [25]:
features = ["acousticness", "danceability","energy","loudness", "tempo","valence"]

artists = normalized_songs.artist.unique()
means = normalized_songs[['artist'] + features].groupby('artist').agg(["mean"])
means.columns = means.columns.get_level_values(0)
std = normalized_songs[['artist'] + features].groupby('artist').agg(["std"])
std.columns = std.columns.get_level_values(0)
var = normalized_songs[['artist'] + features].groupby('artist').agg(["var"])
var.columns = var.columns.get_level_values(0)
for artist in artists:
    test = pd.DataFrame()
    test['mean'] = means.loc[artist].transpose()
    test['std'] = std.loc[artist].transpose()
    test['var'] = var.loc[artist].transpose()
    test['ci_ub'] = test['mean'] + test['std']
    test['ci_lb'] = test['mean'] - test['std']
    test.ci_ub = test.ci_ub.map(lambda ub: min(1, ub))
    test.ci_lb = test.ci_lb.map(lambda lb: max(0, lb))
    test.reset_index().rename(columns={'index':'group'}).to_csv(f'../data/{artist}_mean_std.csv')

In [26]:
test

,mean,std,var,ci_ub,ci_lb
acousticness,0.179568,0.256644,0.065866,0.436212,0.000000
danceability,0.631123,0.137992,0.019042,0.769115,0.493132
energy,0.667394,0.169892,0.028863,0.837286,0.497502
loudness,0.219185,0.060253,0.003630,0.279437,0.158932
tempo,0.579054,0.141977,0.020157,0.721031,0.437077
valence,0.459274,0.227902,0.051939,0.687176,0.231372
